# **Supermarket Sales Analysis: Binary Brogrammers Project**

### Project Overview
This supermarket sales analysis project is crucial to gaining a competitive advantage in today's dynamic retail landscape. Analyzing customer behaviour allows for informed decision-making, leading to **increased sales and profitability**. It needs to be more accurate in understanding buying patterns that could result in missed opportunities and loss of revenue. This project has the potential to provide insights that can be used to improve supermarket operations, enhance customer experience, and drive profitability.

### Business Problems
1. What are the most selling products and wich are the declining sales products?
2. Are there peak sales time (days, month, season)?
3. What is the most preferred payment method?
4. What shopping mall has the highest sales?
5. What Age group has the highest sales?

### Project Goals
1. Identify the most selling products and the declining sales products.
2. Identify peak sales time in terms of days, months, and seasons.
3. Determine the most preferred payment method.
4. Identify the shopping mall with the highest sales.
5. Identify the age group with the highest sales.

## Import Libraries

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load Data

In [11]:
shopdat = pd.read_csv('../data/customer_shopping_data.csv')
print(f'Data Rows: {shopdat.shape[0]}\nData Columns: {shopdat.shape[1]}')

Data Rows: 99457
Data Columns: 10


## First look at the data

In [12]:
print(shopdat.info())
print(f'\nTotal Duplicated Rows:\n{shopdat.duplicated().sum()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_no      99457 non-null  object 
 1   customer_id     99457 non-null  object 
 2   gender          99457 non-null  object 
 3   age             99457 non-null  int64  
 4   category        99457 non-null  object 
 5   quantity        99457 non-null  int64  
 6   price           99457 non-null  float64
 7   payment_method  99457 non-null  object 
 8   invoice_date    99457 non-null  object 
 9   shopping_mall   99457 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB
None

Total Duplicated Rows:
0


In [13]:
shopdat.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon


Summary findings from the first look of the dataset:
- Seems like `price` column is better renamed to total price because as we can see at the 3rd row, the price for 1 clothing is 300.08 while on the first row the clothing price for 5 quantity is 1500.40 which is the total price for 5 quantity. So, I will rename the column to total_price.
- We can extract `invoice_date` to get the year, month, day, weekend,seasons
- We can group `age` into age groups based on the `age` distribution itself


## Feature Engineering

In [14]:
# Create a copy of the original dataset
shopdat_fe = shopdat.copy()

# Rename 'price' column to 'total_price'
shopdat_fe.rename(columns={'price': 'total_price'}, inplace=True)

# Create a new column 'unit_price' by dividing 'total_price' by 'quantity' 
shopdat_fe['unit_price'] = shopdat_fe['total_price'] / shopdat_fe['quantity']

# Convert 'invoice_date' to datetime format and extract the date
shopdat_fe['invoice_date'] = pd.to_datetime(shopdat_fe['invoice_date'], format='%d/%m/%Y').dt.date

# Create new columns for the year, month, and day of the transaction
shopdat_fe['year_trx'] = pd.DatetimeIndex(shopdat_fe['invoice_date']).year
shopdat_fe['month_trx'] = pd.DatetimeIndex(shopdat_fe['invoice_date']).month_name().str.slice(stop=3)
shopdat_fe['day_trx'] = pd.DatetimeIndex(shopdat_fe['invoice_date']).day_name()

# Create a new column 'weekend_trx' that indicates whether the transaction occured on a weekend
shopdat_fe['weekend_trx'] = shopdat_fe['day_trx'].apply(lambda x: 'Yes' if x in ['Saturday', 'Sunday'] else 'No')

# Define the months for each season
se_winter = ['Jan', 'Feb', 'Dec']
se_spring = ['Mar', 'Apr', 'May']
se_summer = ['Jun', 'Jul', 'Aug']
se_autumn = ['Sep', 'Oct', 'Nov']

# Create a new column 'seasons' that indicates the season in which the transaction occured
shopdat_fe['seasons'] = shopdat_fe['month_trx'].apply(lambda x: 'Winter' if x in se_winter else 'Spring' if x in se_spring else 'Summer' if x in se_summer else 'Autumn')

# Create a new column 'age_group' that categorizes the customers into age groups
# Define the bin edges according to the quartiles
bins = [18, 30, 43, 56, 69]

# Define the bin labels
labels = ['18-29', '30-42', '43-55', '56-69']

shopdat_fe['age_group'] = pd.cut(shopdat_fe['age'], bins=bins, labels=labels, include_lowest=True)

In [15]:
# Display 3 random rows of the dataset after feature engineering
shopdat_fe.sample(3)

,invoice_no,customer_id,gender,age,category,quantity,total_price,payment_method,invoice_date,shopping_mall,unit_price,year_trx,month_trx,day_trx,weekend_trx,seasons,age_group
43451,I136999,C203343,Female,61,Food & Beverage,1,5.23,Debit Card,2021-05-28,Metropol AVM,5.23,2021,May,Friday,No,Spring,56-69
32335,I264612,C524148,Male,33,Clothing,5,1500.40,Credit Card,2021-03-06,Mall of Istanbul,300.08,2021,Mar,Saturday,Yes,Spring,30-42
78784,I101629,C102585,Male,41,Cosmetics,3,121.98,Cash,2021-09-25,Mall of Istanbul,40.66,2021,Sep,Saturday,Yes,Autumn,30-42
